##  Curves (x–y diagnostics)

These “curves” visualize the **relationship between two model variables** (x on the horizontal axis, y on the vertical) after you’ve applied your **time filters**, **spatial scope** (domain / region / station), and **depth selection**. They answer questions like:

* **How does Y respond to X?** e.g., *chl vs PAR*, *phyto vs temp*, *DOC vs mixed-layer depth*.
* **Is there a threshold, optimum, or saturation?** (e.g., light limitation at low PAR; nutrient saturation at high DIN.)
* **How different are regions or stations?** (Plot multiple curves with the same x/y but different scopes.)
* **How does the relationship change with depth or season?** (Use different depth selections or time windows.)

---

#### Defining specs (what to plot)

`plot_curves(specs=[...], ds=..., groups=...)` takes a list of spec dictionaries—one per curve. Each spec tells the function what to compute and how to render it.

##### Required
```python
x: str # variable/expression for the X-axis

y: str # variable/expression for the Y-axis

#Common (optional but useful)

name: str # legend label for this curve

filters: dict # — time/predicate filters
    months: [6,7,8] # Example JJA
    years: [2018] # Example 1 year
    start / end: "YYYY-MM-DD"
    where: str # boolean expression mask (e.g., "PAR > 0", or a named expression in groups)

depth: one of
    "surface", "bottom", "depth_avg"
    a sigma level (float, e.g., 0.5)
    absolute-z dict, e.g., {"z_m": -10} (10 m below surface)

scope: dict # where to sample
    {"region": (name, spec)} # polygon from shapefile or boundary CSV
    {"station": (name, lat, lon)} # nearest column
    {} — domain # (default)

Choose one rendering mode

bin: {"x_bins": 40, "agg": "median"|"mean"|"p90", "min_count": 10, "iqr": True} # draws binned median with optional IQR band
scatter: {"s": 4, "alpha": 0.15} # plots all samples as a semi-transparent cloud

style: matplotlib overrides (e.g., {"color": "C3"})

aliases (optional): per-spec name remaps, e.g. {"PAR": "light_parEIR"}
```
##### About names & expressions

x/y (and filters.where) support GROUPS and algebra (e.g., "P1_Chl + P2_Chl").

Variable lookup is tolerant to case/underscores (chl_total, ChlTotal, chl-total all match if present).

---

#### Auto file naming

If you don’t pass stem=..., the function auto-builds a filename tag from:

Scope: Domain | Region-<Name> | Station-<Name> | MultiScope

Depth: e.g., surface, bottom, depth_avg, sigma0.5, z-10m, or MixedDepth

Time: e.g., JJA, 2018, 2018-04–2018-10, or MixedTime

Content: <X>_vs_<Y> (+ Ncurves if multiple specs)

    Figures are written under:
    FIG_DIR/<basename(BASE_DIR)>/curves/
    …unless overridden via FVCOM_PLOT_SUBDIR.

---

#### Two ways of drawing the relationship

* **Binned median + IQR (robust trend):**
  X is split into equal-width bins. For each bin we compute the **median** Y (the central tendency) and the **IQR band** (25–75th percentiles) to show spread.

  * **When to use:** you want a **clean functional shape** (less noise), and a sense of variability that’s robust to outliers.
  * **What it means:** the line traces the *typical* Y for a given X; the shaded region is the central half of outcomes at that X.

* **Raw scatter (cloud of points):**
  Every time×space sample (after filters/masks) is plotted.

  * **When to use:** you want to see **full dispersion**, multi-modal patterns, or rare extremes the median might hide.
  * **What it means:** density and spread of points reflect how often combinations of (X,Y) occur in your filtered subset.

> Tip: Combine both—plot a **binned curve** for the backbone and **scatter** (semi-transparent) underneath for context.

---

#### How to *read* the curves

* **Monotonic rise or fall:** suggests **limitation** or **inhibition** (e.g., phyto rising with PAR up to saturation).
* **Plateau (saturation):** beyond a certain X, **another** factor likely limits Y.
* **Hump-shaped (optimum):** indicates **peak performance** at intermediate X (temperature, for instance).
* **Wide IQR band:** strong **spatial/temporal heterogeneity**, unresolved drivers, or mixed regimes within your filter.
* **Split/looping clouds:** potential **regime shifts**, **seasonality**, or **hysteresis**—narrow your time window or add a `where` predicate.


#### Good practice & caveats

* **Keep units consistent** and label axes clearly (use `x_label`/`y_label` if your expressions are verbose).
* **Mind bin support:** if many bins have **low sample counts**, the curve can wiggle; increase `min_count` or reduce `x_bins`.
* **Avoid mixing regimes** unintentionally—narrow time windows or add a `where` clause to focus the story.
* **Compare like with like:** when contrasting regions/stations, use the **same filters/depth** so differences are interpretable.
* **Consider log scales** (set in your plotting style) if X or Y spans orders of magnitude.
* **Interpret IQR correctly:** it’s the **middle 50%**; large bands do not necessarily mean noise—they may indicate **real heterogeneity**.



In [ ]:
# Setup

BASE_DIR = "/data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var"
FILE_PATTERN = "erie_00??.nc"
FIG_DIR = "/data/proteus1/scratch/moja/projects/Lake_Erie/fviz-plots/"


STATIONS = [
    ("WE12", 41.90, -83.10),
    ("WE13", 41.80, -83.20),
]

REGIONS = [
    ("Central", {"shapefile": "../data/shapefiles/central_basin_single.shp"}),
    ("East", {"shapefile": "../data/shapefiles/east_basin_single.shp"}),
    ("West", {"shapefile": "../data/shapefiles/west_basin_single.shp"}),
]

GROUPS = {
    # Aliases (nice short names you’ll use in specs)
    "PAR": "light_parEIR",
    "DIN": "N3_n + N4_n",
    # Composites (elementwise sums)
    "chl_total": "P1_Chl + P2_Chl + P4_Chl + P5_Chl",
    "phyto_c_total": "P1_c  + P2_c  + P4_c  + P5_c",
    # Derived metrics (safe algebra; add epsilons to avoid divide-by-zero)
    "P5_spec_prod": "P5_Cfix / (P5_c + 1e-12)",
    # Predicates (boolean expressions you can reuse in `filters.where`)
    "PAR_pos": "light_parEIR > 0",
}

from fvcomersemviz.io import load_from_base
from fvcomersemviz.utils import out_dir, file_prefix
from fvcomersemviz.plot import (
    info,
    bullet,
    kv,
    list_files,
    summarize_files,
    print_dataset_summary,
    ensure_paths_exist,
)


from IPython.display import display, Image, SVG
from pathlib import Path

bullet("\nStations (name, lat, lon):")
for s in STATIONS:
    bullet(f"• {s}")

bullet("\nRegions provided:")
for name, spec in REGIONS:
    bullet(f"• {name}: {spec}")
ensure_paths_exist(REGIONS)

#  Discover files
info(" Discovering files")
files = list_files(BASE_DIR, FILE_PATTERN)
summarize_files(files)
if not files:
    print("\nNo files found. Exiting.")
    sys.exit(2)

#  Load dataset
info(" Loading dataset (this may be lazy if Dask is available)")
ds = load_from_base(BASE_DIR, FILE_PATTERN)
bullet("Dataset loaded. Summary:")
print_dataset_summary(ds)

# Where figures will go / filename prefix
out_folder = out_dir(BASE_DIR, FIG_DIR)
prefix = file_prefix(BASE_DIR)
kv("Figure folder", out_folder)
kv("Filename prefix", prefix)

In [ ]:
# --- Curves (x–y) ---


from fvcomersemviz.plots.curves import plot_curves


# --- Curves (x–y diagnostics) reference ---

# plot_curves

# Full argument reference for plot_curves(...)
# Renders one figure containing one or more “curves” describing y vs x relationships,

# def plot_curves(
#     specs: Sequence[Dict[str, Any]],                # Plot_curves requires the user to build a specs dictionary (one dict per curve).
#                                                     # Each spec supports:
#                                                     #   {
#                                                     #     "name": "label for legend",
#                                                     #     "x": "<expr or alias>",                 # resolvable via `groups` and tolerant names
#                                                     #     "y": "<expr or alias>",
#                                                     #     "filters": {                           # optional time/predicate filters
#                                                     #        "months": [6,7,8], "years": [2018],
#                                                     #        "start": "YYYY-MM-DD", "end": "...",
#                                                     #        "where": "<boolean expr>"            # e.g., "light_parEIR > 0"
#                                                     #     },
#                                                     #     "depth": "surface"|"bottom"|"depth_avg"|float sigma|{"z_m": -10},
#                                                     #     "scope": {                              # choose exactly one or none:
#                                                     #        "region": (name, spec)               # e.g., ("Central", {"shapefile": ".../central.shp"})
#                                                     #        # or
#                                                     #        "station": (name, lat, lon)          # nearest-node column extract
#                                                     #     },
#                                                     #     "bin": {                                # to draw a robust trend line
#                                                     #        "x_bins": 40, "agg": "median"|"mean"|"pXX",
#                                                     #        "min_count": 10, "iqr": True         # show IQR band if True
#                                                     #     },
#                                                     #     # If "bin" omitted → raw scatter
#                                                     #     "scatter": {"alpha": 0.15, "s": 4},     # when plotting scatter
#                                                     #     "style": {...},                         # Matplotlib kwargs (color, lw, etc.)
#                                                     #     "aliases": {"PAR": "light_parEIR"}      # optional per-spec alias map
#                                                     #   }
#     *,
#     ds: xr.Dataset,                                  # FVCOM–ERSEM dataset (already opened/combined).
#     groups: Optional[Dict[str, Any]] = None,         # Global alias/composite/derived expressions used by specs, e.g.:
#                                                       # {"chl_total": "P1_Chl + P2_Chl + P4_Chl",
#                                                       #  "DIN": "N3_n + N4_n"}
#     # --- axes labels / legend ---
#     xlabel: Optional[str] = None,                    # Force x-axis label; default picks from first spec ("x_label" or "x").
#     ylabel: Optional[str] = None,                    # Force y-axis label; default picks from first spec ("y_label" or "y").
#     show_legend: bool = True,                        # Toggle legend.
#     legend_outside: bool = True,                     # If True, place legend outside (right); else use "best".
#     legend_fontsize: int = 8,                        # Legend font size.
#     verbose: bool = False,                           # Print resolution/filters/where errors in a tolerant way.
#     # --- figure creation + saving (ALWAYS saves) ---
#     base_dir: str,                                   # Model run root; used by file_prefix() and output path builder.
#     figures_root: str,                               # Root folder where figures are written (package will create subfolders).
#     stem: Optional[str] = None,                      # Optional filename stem override. If None, a stem is auto-built from:
#                                                       #   ScopeTag (Domain|Region-<N>|Station-<N>|MultiScope),
#                                                       #   DepthTag (surface|bottom|depth_avg|z-XXm|AllDepth|MixedDepth),
#                                                       #   TimeLabel (JJA|2018|YYYY-MM–YYYY-MM|AllTime|MixedTime),
#                                                       #   Content ("<X>_vs_<Y>" and "Ncurves" if >1 spec).
#     dpi: int = 150,                                  # Output PNG resolution.
#     figsize: Tuple[float, float]] = (7.2, 4.6),      # Figure size in inches.
#     constrained_layout: bool = True,                 # Use constrained layout when creating the figure.
# ) -> str:
#     pass  # Internals:
#           # 1) For each spec: filter time → apply scope → select depth → resolve x/y (with tolerant names, groups, aliases)
#           #    → optional 'where' predicate → align/flatten → produce binned stats (median+IQR) or scatter payload.
#           # 2) Create a figure and axes; draw each curve with distinct colors from the rc cycle.
#           # 3) Auto-label axes if not provided; place legend (outside by default).
#           # 4) Build output folder via fvcomersemviz.utils.out_dir(). Subdir behavior:
#           #      - If FVCOM_PLOT_SUBDIR is set (e.g., "project" or ""), it is respected.
#           #      - Else defaults to "curves", so files go under .../<basename(BASE_DIR)>/curves/.
#           # 5) Build filename:
#           #      <prefix>__Curves__<ScopeTag>__<DepthTag>__<TimeLabel>__<Content>.png
#           #    or, if `stem` provided: <prefix>__Curves__<stem>.png
#           # 6) Save the PNG and return the full path.

# Output path pattern:
#   <figures_root>/<basename(base_dir)>/<subdir>/
#     <prefix>__Curves__<ScopeTag>__<DepthTag>__<TimeLabel>__<Content>.png
#
# where:
#   <prefix>     = file_prefix(base_dir)
#   <subdir>     = env FVCOM_PLOT_SUBDIR if set; otherwise "curves"
#   <ScopeTag>   = Domain | Region-<Name> | Station-<Name> | MultiScope
#   <DepthTag>   = surface | bottom | depth_avg | z-10m | AllDepth | MixedDepth
#   <TimeLabel>  = built from months/years/start/end (e.g., JJA, 2018, 2018-04–2018-10, AllTime, MixedTime)
#   <Content>    = "<X>_vs_<Y>" from the first spec plus "Ncurves" if multiple curves are shown
#
# Notes:
# - If a spec requests "bin", a robust median curve is drawn with optional IQR shading; otherwise raw scatter is used.
# - Variable resolution is tolerant to case/underscores and can evaluate algebraic expressions via `groups`.
# - A failing 'where' expression is safely ignored with a verbose note if `verbose=True`.
# - The function always saves and returns the output PNG path; you do not need to manage axes or saving yourself.


#    Key fields:
#      - name: legend label for the curve
#      - x, y: variables/expressions (can use GROUPS keys like "PAR", "chl_total", etc.)
#      - filters: months/years/start/end + optional 'where' (can use GROUPS predicates like "PAR_pos")
#      - depth: "surface" | "bottom" | "depth_avg" | float sigma | {"z_m": -10}
#      - scope: {"region": (name, spec)} | {"station": (name, lat, lon)} | {}
#      - Choose ONE of:  bin={...}  OR  scatter={...}
#      - style: optional matplotlib kwargs (color/linestyle/marker/etc.)

# Example 1 — Region vs Region (binned median + IQR), surface, JJA 2018, daylight only

# Example spec
specs_light_chl = [
    {
        "name": "Central",
        "x": "PAR",  # alias -> light_parEIR
        "y": "chl_total",  # composite chlorophyll
        "filters": {"months": [6, 7, 8], "years": [2018], "where": "PAR_pos"},
        "depth": "surface",
        "scope": {"region": REGIONS[0]},
        "bin": {"x_bins": 40, "agg": "median", "min_count": 20, "iqr": True},
        "style": {"color": "C0"},
        "x_label": "PAR (EIR)",
        "y_label": "Total chlorophyll",
    },
    {
        "name": "East",
        "x": "PAR",
        "y": "chl_total",
        "filters": {"months": [6, 7, 8], "years": [2018], "where": "PAR_pos"},
        "depth": "surface",
        "scope": {"region": REGIONS[1]},
        "bin": {"x_bins": 40, "agg": "median", "min_count": 20, "iqr": True},
        "style": {"color": "C3"},
    },
]

# Plot
plot_curves(
    specs=specs_light_chl,
    ds=ds,
    groups=GROUPS,
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    dpi=150,
)

# Example 2 — Domain, depth-avg, Apr–Oct 2018 (binned)

# Example spec
specs_temp_phyto = [
    {
        "name": "Domain",
        "x": "temp",
        "y": "phyto_c_total",
        "filters": {"months": [4, 5, 6, 7, 8, 9, 10], "years": [2018]},
        "depth": "depth_avg",
        "scope": {},  # domain
        "bin": {"x_bins": 32, "agg": "median", "min_count": 20, "iqr": True},
        "style": {"color": "C2"},
        "x_label": "Temperature (°C)",
        "y_label": "Total phytoplankton C",
    }
]

# Plot
plot_curves(
    specs=specs_temp_phyto,
    ds=ds,
    groups=GROUPS,
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    dpi=150,
)

# Example 3 — Domain, depth-avg, Apr–Oct 2018 (scatter cloud)

# Example spec
specs_temp_prod_scatter = [
    {
        "name": "Domain",
        "x": "temp",
        "y": "P5_spec_prod",  # derived metric from GROUPS
        "filters": {"months": [4, 5, 6, 7, 8, 9, 10], "years": [2018]},
        "depth": "depth_avg",
        "scope": {},
        "scatter": {"s": 3, "alpha": 0.12},
        "style": {"marker": ".", "linewidths": 0},
        "x_label": "Temperature (°C)",
        "y_label": "P5 specific production (Cfix / C)",
    }
]

# Plot
plot_curves(
    specs=specs_temp_prod_scatter,
    ds=ds,
    groups=GROUPS,
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    dpi=150,
)

# In order to show both the binned median (backbone) and the scatter cloud on the same graph,
# we include two specs in the same list: one with "scatter" for the raw points,
# and one with "bin" for the aggregated median + IQR curve.

# Example 4: Binned backbone + scatter context (same x/y, same filters/scope/depth) ---

specs_par_chl_backbone = [
    # 1) Scatter cloud for context (drawn first → under the line)
    {
        "name": "All points",
        "x": "PAR",  # alias from GROUPS → light_parEIR
        "y": "chl_total",  # composite from GROUPS
        "filters": {"months": [6, 7, 8], "years": [2018], "where": "PAR_pos"},
        "depth": "surface",
        "scope": {},  # domain
        "scatter": {"s": 6, "alpha": 0.08},
        "style": {"marker": ".", "linewidths": 0, "color": "red"},
        "x_label": "PAR (EIR)",
        "y_label": "Total chlorophyll",
    },
    # 2) Binned median + IQR “backbone” (drawn second → on top)
    {
        "name": "Median (IQR)",
        "x": "PAR",
        "y": "chl_total",
        "filters": {"months": [6, 7, 8], "years": [2018], "where": "PAR_pos"},
        "depth": "surface",
        "scope": {},
        "bin": {"x_bins": 40, "agg": "median", "min_count": 20, "iqr": True},
        "style": {"color": "blue", "lw": 2},
    },
]

# Plot
plot_curves(
    specs=specs_par_chl_backbone,
    ds=ds,
    groups=GROUPS,
    base_dir=BASE_DIR,
    figures_root=FIG_DIR,
    dpi=150,
    legend_outside=True,
)


print(" Curve examples completed. Figures saved under:", FIG_DIR)

# ---- Inline preview: show curves images from this run ----
RUN_ROOT = Path(FIG_DIR) / Path(BASE_DIR).name  # <FIG_DIR>/<basename(BASE_DIR)>
CURVES_DIR = RUN_ROOT / "curves"
search_root = CURVES_DIR if CURVES_DIR.exists() else RUN_ROOT

files = sorted(
    list(search_root.rglob("*.png")) + list(search_root.rglob("*.svg")),
    key=lambda p: p.stat().st_mtime,
)

if not files:
    print(f"No curve images found under {search_root}")
else:
    N = 8
    print(f"Showing the latest {min(N, len(files))} curve plot(s) from {search_root}:")
    for p in files[-N:]:
        print("•", p.relative_to(RUN_ROOT))
        if p.suffix.lower() == ".svg":
            display(SVG(filename=str(p)))
        else:
            display(Image(filename=str(p)))